In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from torch.optim.lr_scheduler import StepLR

from sklearn.metrics import roc_auc_score

import numpy as np
import pandas as pd
import copy
from time import time


In [2]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
housing_df = pd.DataFrame(data=housing.data, columns=housing.feature_names)
housing_df['target'] = housing.target

In [3]:
x = housing_df.drop('target', axis=1)
y = housing_df[['target']]

X, X_test, Y, y_test = train_test_split(x, y, test_size=0.1, random_state=2137)

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.1, random_state=2137)

X_train = torch.from_numpy(X_train.to_numpy().astype('float32'))
X_val = torch.from_numpy(X_val.to_numpy().astype('float32'))
X_test = torch.from_numpy(X_test.to_numpy().astype('float32'))
y_train = torch.from_numpy(y_train.to_numpy().reshape(-1, 1).astype('float32'))
y_val = torch.from_numpy(y_val.to_numpy().reshape(-1, 1).astype('float32'))
y_test = torch.from_numpy(y_test.to_numpy().reshape(-1, 1).astype('float32'))

batch_size=1024
dataset_train = torch.utils.data.TensorDataset(X_train, y_train)
dataset_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_val = torch.utils.data.TensorDataset(X_val, y_val)
dataset_val = torch.utils.data.DataLoader(dataset_val, batch_size=32, shuffle=False)

dataset_test = torch.utils.data.TensorDataset(X_test, y_test)
dataset_test = torch.utils.data.DataLoader(dataset_test, batch_size=32, shuffle=False)


In [183]:
class DNN_model(nn.Module):
    def __init__(self, input_size):
        super(DNN_model, self).__init__()
        self.layer1 = nn.Linear(input_size, 128)
        self.layer2 = nn.Linear(128, 64)
        #self.layer3 = nn.Linear(32, 16)
        self.layer4 = nn.Linear(64, 1)
        #self.dropout1 = nn.Dropout(0.1)
        #self.dropout2 = nn.Dropout(0.1)
        #self.dropout3 = nn.Dropout(0.1)
        self.act = nn.Sigmoid()

    def forward(self, x):
        x = self.act(self.layer1(x))
        #x = self.dropout1(x)
        x = self.act(self.layer2(x))
        #x = self.dropout2(x)
        #x = F.relu(self.layer3(x))
        #x = self.dropout3(x)
        x = self.layer4(x)
        return x


In [184]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [185]:
model = DNN_model(len(dataset_train.dataset[0][0]))

In [186]:
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-6)
scheduler = StepLR(optimizer, step_size=40, gamma=0.2)
criterion=nn.MSELoss()

In [187]:
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    
    start_time = time()
    for batch_idx, (data, target) in enumerate(train_loader):   
        optimizer.zero_grad()   # .backward() accumulates gradients
        data = data.to(device)
        target = target.to(device) # all data & model on same device

        outputs = model(data)
        loss = criterion(outputs.squeeze(), target.squeeze())
        running_loss += loss.item()

        loss.backward()
        optimizer.step()
    
    end_time = time()
    
    running_loss /= len(train_loader)
    print('Training Loss: ', running_loss, 'Time: ',end_time - start_time, 's')
    return running_loss

def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()

        running_loss = 0.0

        Y_pred, Y = [], []
        for batch_idx, (data, target) in enumerate(test_loader):   
            data = data.to(device)
            target = target.to(device)

            outputs = model(data)
            loss = criterion(outputs.squeeze(), target.squeeze()).detach()
            running_loss += loss.item()

            pred = outputs.cpu().detach()

            Y_pred.append(pred.squeeze())
            Y.append(target.squeeze().cpu())

        running_loss /= len(test_loader)
        Y_pred = torch.cat(Y_pred)
        Y = torch.cat(Y)
        r2 = r2_score(Y, Y_pred)
        print("Testing r2_score : "+str(i)+" :",r2)
        print('Testing Loss: ', running_loss)
        return running_loss, r2


In [188]:
n_epochs = 100
for i in range(n_epochs):
    print("epoch no:", i)
    train_loss = train_epoch(model, dataset_train, criterion, optimizer)
    test_loss, test_acc = test_model(model, dataset_val, criterion)
    scheduler.step()


epoch no: 0
Training Loss:  2.3642242936527027 Time:  0.242279052734375 s
Testing r2_score : 0 : -0.018066223630313738
Testing Loss:  1.3011857483346583
epoch no: 1
Training Loss:  1.3736993565278894 Time:  0.21291017532348633 s
Testing r2_score : 1 : -0.03809797038193219
Testing Loss:  1.3373406862808486
epoch no: 2
Training Loss:  1.3585059011683744 Time:  0.2906637191772461 s
Testing r2_score : 2 : 0.0015621487941418488
Testing Loss:  1.2791948065919392
epoch no: 3
Training Loss:  1.34118749814875 Time:  0.2106339931488037 s
Testing r2_score : 3 : 0.002846576306152815
Testing Loss:  1.2779142684855704
epoch no: 4
Training Loss:  1.340509379611296 Time:  0.21464276313781738 s
Testing r2_score : 4 : 0.004256990280688933
Testing Loss:  1.276786600129079
epoch no: 5
Training Loss:  1.3404129042344934 Time:  0.22380876541137695 s
Testing r2_score : 5 : 0.007405852736923402
Testing Loss:  1.2724721613576857
epoch no: 6
Training Loss:  1.3240591638228472 Time:  0.30002617835998535 s
Testin

In [174]:
def cal_accuracy(y_test, y_pred):
    print("r2_score : ", r2_score(y_test, y_pred))
    print("MSE :", mean_squared_error(y_test, y_pred))

In [176]:
model.eval()
Y_pred, Y = [], []
for features, targets in dataset_test:
  y_pred = model(features).cpu().detach()
  Y_pred.append(y_pred.squeeze())
  Y.append(targets.squeeze())
Y_pred = torch.cat(Y_pred)
Y = torch.cat(Y)
print("r2 of model :", r2_score(np.array(Y),np.array(Y_pred)))
print("MSE of the model :", criterion(Y, Y_pred))

r2 of model : 0.6437082590688332
MSE of the model : tensor(0.4624)


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
